In [1]:
from openai import OpenAI
import os
import logging
from jinja2 import Environment, select_autoescape
import json
from miniautogen.pipeline.pipeline import PipelineComponent
from miniautogen.llms.llm_client import OpenAIClient, LiteLLMClient
from dotenv import load_dotenv
load_dotenv()

api_key=os.getenv('OPENAI_API_KEY')
openai_client = OpenAIClient(api_key=api_key)
litellm_client = LiteLLMClient(model="ollama/llama2")

In [2]:
from miniautogen.chat.chat import Chat
from miniautogen.agent.agent import Agent
from miniautogen.chat.chatadmin import ChatAdmin
from miniautogen.pipeline.pipeline import Pipeline
from miniautogen.pipeline.components import (
    UserResponseComponent,
    AgentReplyComponent,
    TerminateChatComponent,
    Jinja2SingleTemplateComponent,
    UpdateNextAgentComponent,
    NextAgentMessageComponent,
    LLMResponseComponent
)

template_str = """
[
  {"role": "system", "content": "{{ agent.role }}"}{% for message in messages %},
  {% if message.sender_id == agent.agent_id %}
    {"role": "assistant", "content": {{ message.message | tojson | safe }}}
  {% else %}
    {"role": "user", "content": {{ message.message | tojson | safe }}}
  {% endif %}
{% endfor %}
]
"""

# Cria uma instância do componente com o template
jinja_component = Jinja2SingleTemplateComponent()
jinja_component.set_template_str(template_str)

UpdateNextAgent = UpdateNextAgentComponent()
UpdateNextAgent.set_next_agent_id("Bruno")

NextAgentMessage = NextAgentMessageComponent()
NextAgentMessage.set_alternative_next(UpdateNextAgent)

# Configuração dos Pipelines
pipeline_user = Pipeline([UserResponseComponent()])
pipeline_jinja = Pipeline([jinja_component, LLMResponseComponent(litellm_client)])
pipeline_admin = Pipeline(
    [NextAgentMessage, AgentReplyComponent(), TerminateChatComponent()])

# Setup do ambiente de teste
chat = Chat()
json_data = {
    'agent_id': 'Bruno',
    'name': 'Bruno',
    'role': 'user'
}

# Criação de Agentes
agent1 = Agent.from_json(json_data)
agent1.pipeline = pipeline_user  # Atribuindo o pipeline ao agente

agent2 = Agent("dev", "Carlos", "Python senior Developer")
agent2.pipeline = pipeline_jinja  # Atribuindo o pipeline_llm ao segundo agente


# Adicionando os agentes ao chat
chat.add_agent(agent1)
chat.add_agent(agent2)

# Adição de mensagens de teste ao chat
json_messages = [
    {'sender_id': 'user4', 'message': 'Que bom ouvir isso!'},
    {'sender_id': 'user5', 'message': 'Vamos continuar Bruno.',
        'additional_info': {'topic': 'chat'}},
]
chat.add_messages(json_messages)

# Criação e configuração do ChatAdmin
chat_admin = ChatAdmin("admin", "Admin", "admin_role",
                       pipeline_admin, chat, "manage_chat", 10)

In [3]:
chat_admin.run()

INFO:miniautogen.chat.chatadmin:Chat Admin started.
INFO:miniautogen.chat.chatadmin:Executing round 1
INFO:miniautogen.chat.chatadmin:Executing round 2


Olá dev, quem é voc?E
[{'role': 'system', 'content': 'Python senior Developer'}, {'role': 'user', 'content': 'Que bom ouvir isso!'}, {'role': 'user', 'content': 'Vamos continuar Bruno.'}, {'role': 'user', 'content': 'Olá dev, quem é voc?E'}]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:miniautogen.chat.chatadmin:Executing round 3


Olá, sou um assistente de inteligência artificial treinado para ajudar em várias tarefas, incluindo programação em Python.
Nenhum agente correspondente encontrado na última mensagem.
Próximo agente atualizado para: Bruno


INFO:miniautogen.chat.chatadmin:Chat Admin stopped.
INFO:miniautogen.chat.chatadmin:Chat Admin stopped.


terminate
Encerrando chat...


In [4]:
chat.get_messages().to_dict(orient='records')

[{'id': 1,
  'sender_id': 'user4',
  'message': 'Que bom ouvir isso!',
  'timestamp': Timestamp('2024-01-02 19:24:05.525563'),
  'additional_info': None},
 {'id': 2,
  'sender_id': 'user5',
  'message': 'Vamos continuar Bruno.',
  'timestamp': Timestamp('2024-01-02 19:24:05.527200'),
  'additional_info': {'topic': 'chat'}},
 {'id': 3,
  'sender_id': 'Bruno',
  'message': 'Olá dev, quem é voc?E',
  'timestamp': Timestamp('2024-01-02 19:24:13.705231'),
  'additional_info': None},
 {'id': 4,
  'sender_id': 'dev',
  'message': 'Olá, sou um assistente de inteligência artificial treinado para ajudar em várias tarefas, incluindo programação em Python.',
  'timestamp': Timestamp('2024-01-02 19:24:17.275782'),
  'additional_info': None},
 {'id': 5,
  'sender_id': 'Bruno',
  'message': 'terminate',
  'timestamp': Timestamp('2024-01-02 19:24:29.021759'),
  'additional_info': None}]